In [2]:
import string

def normalize(text: str) -> str:
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return " ".join(text.split()).strip()

def presence_match(test_pairs):
    N = len(test_pairs)
    matches = 0
    for gt, pred in test_pairs:
        gt_words = set(normalize(gt).split())
        pred_words = set(normalize(pred).split())
        if gt_words.issubset(pred_words):
            matches += 1
    return matches * 100 / N if N > 0 else 0.0

def extract_pairs_from_markdown(content: str):
    pairs = []
    gt, pred = None, None
    for line in content.splitlines():
        line = line.strip()
        if line.startswith("**Ground Truth:**"):
            gt = line.replace("**Ground Truth:**", "").strip()
        elif line.startswith("**Retrieved Answer:**"):
            pred = line.replace("**Retrieved Answer:**", "").strip()
        if gt and pred:
            pairs.append((gt, pred))
            gt, pred = None, None
    return pairs


if __name__ == "__main__":
    md_file_path = r"C:\Users\DAIICT D\Desktop\EXTRINSIC\hotpot\14b\hotpot_14b.md"
    with open(md_file_path, "r", encoding="utf-8") as f:
        content = f.read()

    test_pairs = extract_pairs_from_markdown(content)

    presence_score = presence_match(test_pairs)
    print(f"Presence Match Score: {presence_score:.4f} %")


Presence Match Score: 54.2000 %


In [3]:
import string

def normalize(text: str) -> str:
    # Lowercase, remove punctuation, trim extra spaces
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return " ".join(text.split()).strip()

def exact_match(test_pairs):
    N = len(test_pairs)
    matches = 0
    for gt, pred in test_pairs:
        if normalize(gt) == normalize(pred):
            matches += 1
    return matches * 100 / N if N > 0 else 0.0

def extract_pairs_from_markdown(content: str):
    pairs = []
    gt, pred = None, None
    for line in content.splitlines():
        line = line.strip()
        if line.startswith("**Ground Truth:**"):
            gt = line.replace("**Ground Truth:**", "").strip()
        elif line.startswith("**Retrieved Answer:**"):
            pred = line.replace("**Retrieved Answer:**", "").strip()
        if gt and pred:
            pairs.append((gt, pred))
            gt, pred = None, None
    return pairs


if __name__ == "__main__":
    md_file_path = r"C:\Users\DAIICT D\Desktop\EXTRINSIC\hotpot\14b\hotpot_14b.md"
    with open(md_file_path, "r", encoding="utf-8") as f:
        content = f.read()

    test_pairs = extract_pairs_from_markdown(content)

    em_score = exact_match(test_pairs)
    print(f"Exact Match Score: {em_score:.4f} %")

Exact Match Score: 38.0000 %


In [4]:
from typing import List
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re

# ----- Parse markdown input file for answer pairs -----
def load_test_pairs_from_markdown(md_file_path: str):
    with open(md_file_path, "r", encoding="utf-8") as f:
        content = f.read()

    pairs = []
    gt, rt = None, None
    for line in content.splitlines():
        line = line.strip()
        if line.startswith("**Ground Truth:**"):
            gt = line.replace("**Ground Truth:**", "").strip()
        elif line.startswith("**Retrieved Answer:**"):
            rt = line.replace("**Retrieved Answer:**", "").strip()
        if gt and rt:
            pairs.append((gt, rt))
            gt, rt = None, None
    return pairs


# ----- Embedding model -----
model = SentenceTransformer("all-MiniLM-L6-v2")  # lightweight model


def embed_sentences(sentences: List[str]) -> np.ndarray:
    return model.encode(sentences, convert_to_numpy=True, show_progress_bar=False)


# ----- Retrieval Similarity function -----
def retrieval_similarity(retrieved: List[str], gold: List[str]) -> float:
    assert len(retrieved) == len(gold), "Input lists must be of same length"

    retr_embs = embed_sentences(retrieved)
    gold_embs = embed_sentences(gold)

    sims = [
        cosine_similarity(retr_embs[i].reshape(1, -1), gold_embs[i].reshape(1, -1)).item()
        for i in range(len(retrieved))
    ]

    # Debug: print each pair's similarity score
    for idx, score in enumerate(sims, start=1):
        print(f"Pair {idx} similarity: {score:.4f}")

    return float(np.mean(sims))


# ----- MAIN -----
if __name__ == "__main__":
    md_file_path = r"C:\Users\DAIICT D\Desktop\EXTRINSIC\hotpot\14b\hotpot_14b.md"

    test_pairs = load_test_pairs_from_markdown(md_file_path)

    if not test_pairs:
        print("⚠️ No Ground Truth / Retrieved Answer pairs found. Check formatting.")
    else:
        gold_texts = [gt for gt, _ in test_pairs]
        retrieved_texts = [rt for _, rt in test_pairs]

        rs_score = retrieval_similarity(retrieved_texts, gold_texts) * 100.0
        print(f"\nAverage Retrieval Similarity (R-S): {rs_score:.4f} %")


c:\Users\DAIICT D\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pair 1 similarity: 1.0000
Pair 2 similarity: 1.0000
Pair 3 similarity: 1.0000
Pair 4 similarity: 0.4387
Pair 5 similarity: 0.6605
Pair 6 similarity: 0.9698
Pair 7 similarity: 0.2586
Pair 8 similarity: 1.0000
Pair 9 similarity: 0.8165
Pair 10 similarity: 0.8128
Pair 11 similarity: 1.0000
Pair 12 similarity: 0.7972
Pair 13 similarity: 1.0000
Pair 14 similarity: 0.4838
Pair 15 similarity: 1.0000
Pair 16 similarity: 0.6322
Pair 17 similarity: 0.1488
Pair 18 similarity: 0.8137
Pair 19 similarity: 0.6617
Pair 20 similarity: 1.0000
Pair 21 similarity: 0.7994
Pair 22 similarity: 1.0000
Pair 23 similarity: 0.6472
Pair 24 similarity: 0.7358
Pair 25 similarity: 1.0000
Pair 26 similarity: 0.8369
Pair 27 similarity: 0.2071
Pair 28 similarity: 0.1544
Pair 29 similarity: 0.3840
Pair 30 similarity: 0.2289
Pair 31 similarity: 1.0000
Pair 32 similarity: 0.9484
Pair 33 similarity: 0.8128
Pair 34 similarity: 0.2972
Pair 35 similarity: 0.6706
Pair 36 similarity: 0.4282
Pair 37 similarity: 1.0000
Pair 38 si